In [38]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [39]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-02 16:18:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-02 16:18:24 (6.46 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my_project_20_group_3").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [41]:
from pyspark import SparkFiles
url = "https://rural-surge-b1.s3.amazonaws.com/Fixed_Broadband_Deployment_Data__December_2020.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("delimiter", ",").option("header",True).csv(SparkFiles.get("Fixed_Broadband_Deployment_Data__December_2020.csv"))
df.show()

+---------------------+-----------+----------+--------------------+--------------------+--------------------+----------------------+---------------------+-----+----------------------+---------------+--------+--------------------------------------+------------------------------------+--------+
|Logical Record Number|Provider ID|       FRN|       Provider Name|            DBA Name|Holding Company Name|Holding Company Number|Holding Company Final|State|Census Block FIPS Code|Technology Code|Consumer|Max Advertised Downstream Speed (mbps)|Max Advertised Upstream Speed (mbps)|Business|
+---------------------+-----------+----------+--------------------+--------------------+--------------------+----------------------+---------------------+-----+----------------------+---------------+--------+--------------------------------------+------------------------------------+--------+
|              2502903|      69801|0006121354|Wheatland Electri...|Wheatland Broadba...|Wheatland Electri...|         

In [81]:
from pyspark.sql import SparkSession

# the Spark session should be instantiated as follows
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.jars", "postgresql-42.2.14.jar") \
    .getOrCreate()

In [82]:
jdbcDF = spark.read.format("jdbc"). \
options(
         url='jdbc:postgresql://ruralsurge.cu3ibjje45e1.us-east-1.rds.amazonaws.com:5432/rural_surge_db', #jdbc:postgresql://<host>:<port>/<database>
         dbtable='area_table',
         user='root',
         password='########',
         driver='org.postgresql.Driver').\
load()

In [83]:
jdbcDF.printSchema()

root
 |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- tech: string (nullable = true)
 |-- urban_rural: string (nullable = true)
 |-- tribal_non: string (nullable = true)
 |-- speed: double (nullable = true)
 |-- has_0: integer (nullable = true)
 |-- has_1: integer (nullable = true)
 |-- has_2: integer (nullable = true)
 |-- has_3more: integer (nullable = true)
 |-- tech_options: integer (nullable = true)



In [84]:
area_df = spark.read.jdbc(url = "jdbc:postgresql://ruralsurge.cu3ibjje45e1.us-east-1.rds.amazonaws.com:5432/rural_surge_db", 
                     table = "(SELECT * FROM area_table) AS my_table",
                     properties={"user": "root", "password": "Su4g3t0End!", "driver":  "org.postgresql.Driver"})#.createTempView("tbl")

In [85]:
spark.sql('select * from tbl').show() #or use .collect() to get Rows

+-----+---+------+-----------+----------+-----+-----+-----+------+---------+
| type| id|  tech|urban_rural|tribal_non|speed|has_0|has_1| has_2|has_3more|
+-----+---+------+-----------+----------+-----+-----+-----+------+---------+
|state| 20|acfosw|          R|         N|  0.2|    0|    4|     9|   749729|
|state| 20|acfosw|          R|         T|  0.2|    0|    0|     0|     5571|
|state| 20|acfosw|          U|         N|  0.2|    0|    1|     0|  2158410|
|state| 20| acfos|          R|         N|  0.2|    0|   25|     4|   749713|
|state| 20| acfos|          R|         T|  0.2|    0|    0|     0|     5571|
|state| 20| acfos|          U|         N|  0.2|    0|    8|     2|  2158401|
|state| 20| acfow|          R|         N|  0.2| 3219|89226|135256|   522041|
|state| 20| acfow|          R|         T|  0.2|   10|  118|   235|     5208|
|state| 20| acfow|          U|         N|  0.2| 2404|13802|108907|  2033298|
|state| 20| acfsw|          R|         N|  0.2|    0|    4|     9|   749729|

In [86]:
import pyspark.sql.functions as F 
length_df = area_df.withColumn("tech_options", F.length("tech"))
length_df.show(truncate=False)

+-----+---+------+-----------+----------+-----+-----+-----+------+---------+------------+
|type |id |tech  |urban_rural|tribal_non|speed|has_0|has_1|has_2 |has_3more|tech_options|
+-----+---+------+-----------+----------+-----+-----+-----+------+---------+------------+
|state|20 |acfosw|R          |N         |0.2  |0    |4    |9     |749729   |6           |
|state|20 |acfosw|R          |T         |0.2  |0    |0    |0     |5571     |6           |
|state|20 |acfosw|U          |N         |0.2  |0    |1    |0     |2158410  |6           |
|state|20 |acfos |R          |N         |0.2  |0    |25   |4     |749713   |5           |
|state|20 |acfos |R          |T         |0.2  |0    |0    |0     |5571     |5           |
|state|20 |acfos |U          |N         |0.2  |0    |8    |2     |2158401  |5           |
|state|20 |acfow |R          |N         |0.2  |3219 |89226|135256|522041   |5           |
|state|20 |acfow |R          |T         |0.2  |10   |118  |235   |5208     |5           |
|state|20 

In [87]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ruralsurge.cu3ibjje45e1.us-east-1.rds.amazonaws.com:5432/rural_surge_db"
config = {"user":"root", 
          "password": "#######", 
          "driver":"org.postgresql.Driver"}

In [89]:
# Write review_id_df to table in RDS
length_df.write.jdbc(url=jdbc_url, table='area_mod', mode=mode, properties=config)